# Linear Regression Simple 

In [3]:
%matplotlib inline
import torch
import numpy as np
from IPython import display
from matplotlib import pyplot as plt
import random

In [4]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0,1,(num_examples,num_inputs)),dtype=torch.float)
labels = true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels += torch.tensor(np.random.normal(0,0.01,num_examples),dtype=torch.float)

## 读取数据
PyTorch提供了data包来读取数据。由于data常用作变量名，我们将导入的data模块用Data代替。在每一次迭代中，我们将随机读取包含10个数据样本的小批量。

In [5]:
import torch.utils.data as Data
batch_size = 10
# Data.TensorDataset 包装了张量的数据集，即传入张量（第一个维度相同），会通过第一个维度indexing。
data_set = Data.TensorDataset(features,labels)
data_iter = Data.DataLoader(data_set,batch_size,shuffle=True)
data_iter 

## 定义模型
在上一节从零开始的实现中，我们需要定义模型参数，并使用它们一步步描述模型是怎样计算的。当模型结构变得更复杂时，这些步骤将变得更繁琐。其实，PyTorch提供了大量预定义的层，这使我们只需关注使用哪些层来构造模型。下面将介绍如何使用PyTorch更简洁地定义线性回归。

首先，导入torch.nn模块。实际上，“nn”是neural networks（神经网络）的缩写。顾名思义，该模块定义了大量神经网络的层。之前我们已经用过了autograd，而nn就是利用autograd来定义模型。nn的核心数据结构是Module，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。在实际使用中，最常见的做法是继承nn.Module，撰写自己的网络/层。一个nn.Module实例应该包含一些层以及返回输出的前向传播（forward）方法。下面先来看看如何用nn.Module实现一个线性回归模型。

In [6]:
from torch import nn
class LinearNet(nn.Module):
    def __init__(self,n_feature):
        super(LinearNet,self).__init__()
        self.linear = nn.Linear(n_feature,1)
    def forward(self,x):
        y = self.linear(x)
        return y
    

In [7]:
net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


事实上我们还可以用nn.Sequential来更加方便地搭建网络，Sequential是一个有序的容器，网络层将按照在传入Sequential的顺序依次被添加到计算图中。

In [8]:
net1 = nn.Sequential(nn.Linear(num_inputs,1))
print(net1)

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)


In [9]:
net2 = nn.Sequential()
net2.add_module('linear', nn.Linear(num_inputs, 1))
print(net2)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [10]:
from collections import OrderedDict
net3 = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))
print(net3)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


可以通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器。

In [11]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.5395, -0.0671]], requires_grad=True)
Parameter containing:
tensor([0.6659], requires_grad=True)


回顾图3.1中线性回归在神经网络图中的表示。作为一个单层神经网络，线性回归输出层中的神经元和输入层中各个输入完全连接。因此，线性回归的输出层又叫全连接层。

注意：torch.nn仅支持输入一个batch的样本不支持单个样本输入，如果只有单个样本，可使用input.unsqueeze(0)来添加一维。

## 初始化模型参数
在使用net前，我们需要初始化模型参数，如线性回归模型中的权重和偏差。PyTorch在init模块中提供了多种参数初始化方法。这里的init是initializer的缩写形式。我们通过init.normal_将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。偏差会初始化为零。

In [13]:
from torch.nn import init
init.normal_(net1[0].weight,mean = 0,std = 0.01)
init.constant_(net1[0].bias,val = 0)

Parameter containing:
tensor([0.], requires_grad=True)

## 定义损失函数
PyTorch在nn模块中提供了各种损失函数，这些损失函数可看作是一种特殊的层，PyTorch也将这些损失函数实现为nn.Module的子类。我们现在使用它提供的均方误差损失作为模型的损失函数。

In [20]:
loss  = nn.MSELoss()

## 定义优化算法
同样，我们也无须自己实现小批量随机梯度下降算法。torch.optim模块提供了很多常用的优化算法比如SGD、Adam和RMSProp等。下面我们创建一个用于优化net所有参数的优化器实例，并指定学习率为0.03的小批量随机梯度下降（SGD）为优化算法。

In [16]:
import torch.optim as optim
optimizer = optim.SGD(net1.parameters(),lr = 0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


## 训练模型
在使用Gluon训练模型时，我们通过调用optim实例的step函数来迭代模型参数。按照小批量随机梯度下降的定义，我们在step函数中指明批量大小，从而对批量中样本梯度求平均。

In [21]:
num_epochs = 3
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        output = net1(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000247
epoch 2, loss: 0.000140
epoch 3, loss: 0.000086


下面我们分别比较学到的模型参数和真实的模型参数。我们从net获得需要的层，并访问其权重（weight）和偏差（bias）。学到的参数和真实的参数很接近。

In [23]:
dense = net1[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9999, -3.3994]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1998], requires_grad=True)


torch.utils.data模块提供了有关数据处理的工具，torch.nn模块定义了大量神经网络的层，torch.nn.init模块定义了各种初始化方法，torch.optim模块提供了很多常用的优化算法。